In [3]:
import tensorflow as tf
import math as math

data_dir = 'mnist-digits'
batch_size = 32

In [4]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset='training',
  shuffle=True,
  seed=123,
  image_size=(28, 28),
  batch_size=batch_size)

validation_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset='validation',
  seed=123,
  image_size=(28, 28),
  batch_size=batch_size)


Found 42000 files belonging to 10 classes.
Using 33600 files for training.


2023-12-27 20:59:33.484637: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-27 20:59:33.486192: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Found 42000 files belonging to 10 classes.
Using 8400 files for validation.


In [5]:
class_names = train_ds.class_names
print(class_names)
train_num = 33600

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [6]:
def normalize(images, labels):
  images = tf.cast(images, float)
  images /= 255
  return images, labels

#apply this function to every element in train and test data
train_ds = train_ds.map(normalize)
validation_ds = validation_ds.map(normalize)

In [7]:
model = tf.keras.Sequential([
    #tf.keras.layers.RandomHeight(0.3),
    #tf.keras.layers.RandomWidth(0.3),
    tf.keras.layers.RandomZoom(0.3),
    #tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu, input_shape=(28, 28, 3)),
    tf.keras.layers.MaxPooling2D((2,2), strides=2),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2,2), strides=2),
    tf.keras.layers.Conv2D(128, (3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2,2), strides=2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [8]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [9]:
model.fit(train_ds, epochs=2, steps_per_epoch=math.ceil(train_num/batch_size))

Epoch 1/2
1050/1050 [==============================] - 130s 122ms/step - loss: 0.5481 - accuracy: 0.8163
Epoch 2/2
1050/1050 [==============================] - 112s 107ms/step - loss: 0.2204 - accuracy: 0.9308


In [10]:
test_loss, test_accuracy = model.evaluate(validation_ds)

263/263 [==============================] - 2s 7ms/step - loss: 0.1075 - accuracy: 0.9665


In [11]:
model.save('model_save')

INFO:tensorflow:Assets written to: model_save/assets


INFO:tensorflow:Assets written to: model_save/assets
